## CMF 1
Institut für Musikinformatik und Musikwissenschaft – Wintersemester 2025–26
### Woche 06 – Übungen

### Aufgabe 00.

Überlege dir eine Frage zum Inhalt der Vorlesung, z. B. über einen Punkt, der unklar geblieben ist oder über etwas, worüber du gerne mehr wissen möchtest.  

### Aufgabe 01. Gewichtungen in der Levenshtein-Distanz

In [ ]:
import numpy as np
import music21 as m21

Der folgende Code-Block enthält eine Implementierung der Levenshtein-Distanz, wie wir sie in der Vorlesung zur Fugenanalyse genutzt haben.\
Passe diese Implementierung an, indem du dir verschiedene Gewichtungen für die Operationen *Einfügen*, *Löschen* und *Ersetzen* überlegst.\
Welche Gewichtungen könnten für unsere Fugenanalyse sinnvoll sein?

In [130]:
def levenshtein_distance_for_fugues(list1, list2):
    M = len(list1) + 1
    N = len(list2) + 1

    # Matrix zum Abspeichern der Distanzen zwischen allen möglichen Listen-Anfängen
    D = np.zeros([M, N])

    for i in range(M):
        for j in range(N):

            if (i == 0 and j == 0):
                D[i, j] = 0

            elif (i == 0 or j == 0):
                D[i, j] = 1000

            elif list1[i-1] == list2[j-1]:
                D[i, j] = D[i-1, j-1]

            else:
                min_value = min(D[i-1, j-1], D[i-1, j], D[i, j-1])
                D[i, j] = min_value + 1 

    return D[i, j]

In [1]:
def levenshtein_distance_for_fugues(list1, list2):
    M = len(list1) + 1
    N = len(list2) + 1

    # Matrix zum Abspeichern der Distanzen zwischen allen möglichen Listen-Anfängen
    D = np.zeros([M, N])

    for i in range(M):
        for j in range(N):

            if (i == 0 and j == 0):
                D[i, j] = 0

            elif (i == 0 or j == 0):
                D[i, j] = 1000

            elif list1[i-1] == list2[j-1]:
                D[i, j] = D[i-1, j-1]

            # verschiedene Gewichtungen für die Fälle Ersetzen und Einfügen/Löschen
            elif list1[i-1] != list2[j-1]:
                D[i, j] = D[i-1, j-1] + 1

            else:
                min_value = min(D[i-1, j], D[i, j-1])
                D[i, j] = min_value + 2

    return D[i, j]

### Aufgabe 02. Computergestützte Fugenanalyse: Subject-Detection mithilfe der Levenshtein-Distanz.

In [3]:
import numpy as np
import music21 as m21

Die folgenden beiden Code-Blöcke enthalten Funktionen aus der Vorlesung.

In [4]:
def notes_to_diatonic_intervals(note_list=[m21.note.Note("C4"), m21.note.Note("E4"), m21.note.Note("G4")]):

    interval_dictionary = {0: 1, 1:2, 2:2, 3:3, 4:3, 5:4, 6:4.5, 7:5, 8:6, 9:6, 10:7, 11:7, 12:8, 13:9, 14:9, 15:10, 16:10, 17:11,
                                    -1:-2, -2:-2, -3:-3, -4:-3, -5:-4, -6:-4.5, -7:-5, -8:-6, -9:-6, -10:-7, -11:-7, -12:-8, -13:-9, -14:-9, -17:-11}

    num_notes = len(note_list)
    interval_list = []

    for i in range(num_notes - 1):

        current_note = note_list[i].pitch.midi
        next_note = note_list[i+1].pitch.midi

        current_interval = next_note - current_note    
        interval_list.append(current_interval)

    diatonic_interval_list = [interval_dictionary[interval] for interval in interval_list]

    return diatonic_interval_list

In [ ]:
def subject_finder(subject_intervals, voice_intervals):

    len_subject = len(subject_intervals)

    # Wo finden wir ein Thema?
    subject_indices = []
    # Welche Distanzen zum Original-Thema?
    distances = []

    threshold = 4

    for i in range(len(voice_intervals)):

        current_distance = levenshtein_distance_for_fugues(subject_intervals, voice_intervals[i : i+len_subject])

        if current_distance <= threshold:

            subject_indices.append(i)
            distances.append(current_distance)

    return subject_indices, distances

In der Vorlesung haben wir bei der Nutzung der Levenshtein-Distanz zur Erkennung von Fugenthemen nur die Tonhöhen/Intervallen der jeweiligen Stimmen der Fuge berücksichtigt.\
Überlege dir eine Implementierung, die zusätzlich die Notendauern/Rhythmen berücksichtigt. Passe dazu zum Beispiel die Funkion *subject_finder* an.

*Hinweis: Nutze zum Beispiel die Note-Methode .duration.quarterLength*

In [7]:
fugue_score = m21.converter.parse('bach_fugue_cminor.krn')

In [8]:
soprano_notes = list(fugue_score.parts[0].flatten().getElementsByClass(m21.note.Note))
alto_notes = list(fugue_score.parts[1].flatten().getElementsByClass(m21.note.Note))
bass_notes = list(fugue_score.parts[2].flatten().getElementsByClass(m21.note.Note))

soprano_diatonic_intervals = notes_to_diatonic_intervals(soprano_notes)
alto_diatonic_intervals = notes_to_diatonic_intervals(alto_notes)
bass_diatonic_intervals = notes_to_diatonic_intervals(bass_notes)

soprano_note_durations = [note.duration.quarterLength for note in soprano_notes]
alto_note_durations = [note.duration.quarterLength for note in alto_notes]
bass_note_durations = [note.duration.quarterLength for note in bass_notes]

In [9]:
subject_diatonic_intervals = alto_diatonic_intervals[:19]
subject_note_durations = alto_note_durations[:19]

In [10]:
def subject_detection(subject_intervals, subject_note_durations, voice_intervals, voice_note_durations):

    len_subject = len(subject_intervals)

    subject_indices = []
    measured_distances = []

    threshold = 6

    for i in range(len(voice_intervals)):

        current_distance = levenshtein_distance_for_fugues(subject_intervals, voice_intervals[i : i+len_subject])
        current_distance = current_distance + levenshtein_distance_for_fugues(subject_note_durations, voice_note_durations[i : i+len_subject])
        
        if current_distance < threshold:
            subject_indices.append(i)
            measured_distances.append(current_distance)

    return subject_indices, measured_distances

In [11]:
subject_indices, measured_distances = subject_detection(subject_diatonic_intervals, subject_note_durations,
                                                        alto_diatonic_intervals, alto_note_durations)

In [12]:
print("subject indices", subject_indices)
print("measured distances", measured_distances)

subject indices [0, 120]
measured distances [0.0, 2.0]


### Aufgabe 03. Von der Levenshtein-Distanz zur Levenshtein-Similarity.

In [13]:
import numpy as np
import music21 as m21

Versuche, unsere Implementierung der Levenshtein-Distanz so anzupassen, dass aus ihr ein Ähnlichkeits-Maß (Similarity Measure) wird.\\
Das heißt: dort wo die Levenshtein-Distanz hohe Werte liefert (also Werte, die einer hohen *Unähnlichkeit* entsprechen), soll die Levenshtein-Similarity niedrige Werte liefern -- und umgekehrt.\
Nutze die Levenshtein-Similarity zur Erkennung von Fugenthemen, indem du die Implementierung der Funktion *subject_finder* anpasst.

*Hinweis:* Wähle unter anderem einen geeigneten Threshold-Wert.

In [15]:
def levenshtein_similarity_for_fugues(list1, list2):
    M = len(list1) + 1
    N = len(list2) + 1

    # Matrix zum Abspeichern der Ähnlichkeiten zwischen allen möglichen Listen-Anfängen
    S = np.zeros([M, N])

    for i in range(M):
        for j in range(N):

            if (i == 0 and j == 0):
                S[i, j] = 0

            elif (i == 0 or j == 0):
                S[i, j] = -1000

            elif list1[i-1] == list2[j-1]:
                S[i, j] = S[i-1, j-1] + 1

            elif list1[i-1] != list2[j-1]:
                S[i, j] = S[i-1, j-1] - 1

            else:
                min_value = min(S[i-1, j], S[i, j-1])
                S[i, j] = min_value - 3  

    return S[i, j]

In [16]:
def subject_detection2(subject_intervals, subject_note_durations, voice_intervals, voice_note_durations):

    len_subject = len(subject_intervals)

    subject_indices = []
    measured_distances = []

    threshold = 30

    for i in range(len(voice_intervals)):

        current_distance = levenshtein_similarity_for_fugues(subject_intervals, voice_intervals[i : i+len_subject])
        current_distance = current_distance + levenshtein_similarity_for_fugues(subject_note_durations, voice_note_durations[i : i+len_subject])
        
        if current_distance > threshold:
            subject_indices.append(i)
            measured_distances.append(current_distance)

    return subject_indices, measured_distances

In [17]:
subject_indices2, measured_distances2 = subject_detection2(subject_diatonic_intervals, subject_note_durations,
                                                        alto_diatonic_intervals, alto_note_durations)

In [18]:
print("subject indices", subject_indices2)
print("measured distances", measured_distances2)

subject indices [0, 120]
measured distances [38.0, 34.0]
